In [1]:
import pandas as pd
import json

In [11]:
import sys 
import os

In [16]:
common_genes = []
with open(r"C:\Users\Petar\Desktop\PBMC_clustering\PBMC_clustering\log_data.json", "r") as f:
    common_genes = json.load(f)

In [13]:
top_dir = r"C:\Users\Petar\Desktop\Istrazivanje Podataka 2\ip2_seminarski\istrazivanje_podataka2_seminarski\ir_data"

In [22]:
bulk_group_1 = pd.DataFrame(columns=common_genes)
bulk_group_1.head()

,gene_id,ENSG00000125944_HNRNPR,ENSG00000172375_C2CD2L,ENSG00000128789_PSMG2,ENSG00000160209_PDXK,ENSG00000163479_SSR2,ENSG00000122543_OCM,ENSG00000094804_CDC6,ENSG00000168275_COA6,ENSG00000082397_EPB41L3,...,ENSG00000197696_NMB,ENSG00000131100_ATP6V1E1,ENSG00000124614_RPS10,ENSG00000189068_VSTM1,ENSG00000153767_GTF2E1,ENSG00000162909_CAPN2,ENSG00000223685_LINC00571,ENSG00000137601_NEK1,ENSG00000090273_NUDC,ENSG00000019144_PHLDB1


In [26]:
def process_file(filepath, common_genes):
    global bulk_group_1
    print("===========================================")
    print("reading file " + filepath)
    df = pd.read_csv(filepath, usecols=common_genes)
    
    print("dropping redundant cells...")
    df = filter_cells(df)
    
    print("appending to bulk dataframe...")
    bulk_group_1 = bulk_group_1.append(df)

In [27]:
def filter_by_percentage(df, p=1):
    """
    :param df: DataFrame
    :param p: Positive Int
    :return: DataFrame
    returns DataFrame without columns that have less than p% non-zero values
    """
    columns_non_zero_percentages = (df != 0) \
                                        .sum() \
                                        .apply(lambda x: x / df.shape[1] * 100)
    columns_to_keep = (columns_non_zero_percentages > p).values
    return df.iloc[:, columns_to_keep]

In [28]:
def filter_cells(df, sum_of_gene_exprs_lower_limit=1000, number_of_expressed_genes_lower_limit=500):
    """
    :param df: DataFrame
    :sum_of_gene_exprs_lower_limit: Positive Integer
    :number_of_expressed_genes_lower_limit: Positive Integer
    """
    number_of_expressed_genes_per_cell = (df != 0).sum(axis=1)
    sum_of_gene_exprssions_per_cell = df.sum(axis=1)
    cells_to_keep = ((sum_of_gene_exprssions_per_cell > sum_of_gene_exprs_lower_limit) & 
                     (number_of_expressed_genes_per_cell > number_of_expressed_genes_lower_limit)).values
    
    return df.iloc[cells_to_keep, :]

In [30]:
for root, _, files in os.walk(top=top_dir):
    for file in files:
        if file.startswith("GSM2"):
            process_file(os.path.join(root, file), common_genes)

reading file C:\Users\Petar\Desktop\Istrazivanje Podataka 2\ip2_seminarski\istrazivanje_podataka2_seminarski\ir_data\GSM2560245.csv
dropping redundant cells...
appending to bulk dataframe...


c:\python\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


reading file C:\Users\Petar\Desktop\Istrazivanje Podataka 2\ip2_seminarski\istrazivanje_podataka2_seminarski\ir_data\GSM2560246.csv
dropping redundant cells...
appending to bulk dataframe...
reading file C:\Users\Petar\Desktop\Istrazivanje Podataka 2\ip2_seminarski\istrazivanje_podataka2_seminarski\ir_data\GSM2560247.csv
dropping redundant cells...
appending to bulk dataframe...
reading file C:\Users\Petar\Desktop\Istrazivanje Podataka 2\ip2_seminarski\istrazivanje_podataka2_seminarski\ir_data\GSM2560248.csv
dropping redundant cells...
appending to bulk dataframe...
reading file C:\Users\Petar\Desktop\Istrazivanje Podataka 2\ip2_seminarski\istrazivanje_podataka2_seminarski\ir_data\GSM2560249.csv
dropping redundant cells...
appending to bulk dataframe...


In [31]:
bulk_group_1 = filter_by_percentage(bulk_group_1)

In [32]:
bulk_group_1.to_csv("bulk_group_1.csv", index=False)